In [1]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 35.4 MB/s eta 0:00:00


In [2]:
## pdf-Datei anschauen: Die ersten 4 Seiten enthalten Deckblatt + Inhaltsverzeichnis - das kann also übersprungen werden

# Als erstes die Formatinformation extrahieren: Also Text mit Schriftgröße und Dicke der Schrift, um später die Überschriften zu finden und
# zur Gliederung verwenden zu können


import pymupdf as fitz

def extract_text_with_formatting(pdf_path, skip_pages=4):
    doc = fitz.open(pdf_path)
    text_data = []

    for page_num in range(skip_pages, len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]  # Extrahiert Textblöcke

        for block in blocks:
            if block["type"] == 0:  # Textblock
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Speichere Text + Schriftgröße + Fettdruck-Info
                        text_data.append({
                            "text": span["text"],
                            "size": span["size"],  # Schriftgröße
                            "bold": span["flags"] & 2 != 0  # Prüft, ob der Text fett ist
                        })

    return text_data

# PDF-Dateipfad
pdf_path = "/content/Wahlprogramm_FDP.pdf"

# Text mit Formatierungsinformationen extrahieren
text_data = extract_text_with_formatting(pdf_path)

# Beispiel: Zeige die ersten paar Textabschnitte mit Formatierung
for item in text_data[:100]:
    print(f"Text: {item['text']}, Size: {item['size']}, Bold: {item['bold']}")


Text: 4 , Size: 11.0, Bold: False
Text:  , Size: 11.0, Bold: False
Text: I. Präambel: Alles lässt sich ändern , Size: 16.0, Bold: False
Text: Wir alle spüren es: Wie es ist, kann es nicht bleiben. Denn wir können uns nicht damit , Size: 11.0, Bold: False
Text: zufriedengeben, dass Arbeitsplätze und Wohlstand in Gefahr sind. Dass sich Eltern und Großeltern , Size: 11.0, Bold: False
Text: darum sorgen, ob die Bildungs- und Aufstiegschancen ihrer Kinder und Enkelkinder noch so gut sind , Size: 11.0, Bold: False
Text: wie ihre eigenen. Dass innere und äußere Sicherheit herausgefordert sind wie seit langem nicht , Size: 11.0, Bold: False
Text: mehr. Dass die irreguläre Migration immer noch nicht ausreichend unter Kontrolle gebracht ist. Dass , Size: 11.0, Bold: False
Text: wichtige Wertentscheidungen wie Demokratie, Grundrechte und Soziale Marktwirtschaft nicht mehr , Size: 11.0, Bold: False
Text: selbstverständlicher Konsens sind. Dass eine zunehmende Polarisierung die Offenheit und Tolera

In [3]:
## Output anschauen: Normale Schrift ist in Schriftgröße 11, Überschriften sind größer (16), Seitenzahlen stehen in einzelnen Zeilen
## viele leere Zeilen


import pandas as pd


def split_sections(text_data, min_size=11):
    sections = []
    current_section = []

    for item in text_data:
        text = item["text"].strip()

        if not text: #Leere Zeilen überspringen
            continue

        if item["size"] > min_size: # Überschriften überspringen + Abschnitte definieren
            if current_section:
                sections.append(" ".join(current_section).strip())
            current_section = []

        elif text.isdigit() and len(text) <= 4: #Seitenzahlen überspringen
            continue
        else:
            current_section.append(text)

    if current_section:
        sections.append(" ".join(current_section).strip())

    return sections


# Abschnitte ohne Überschriften und leere Zeilen generieren
sections = split_sections(text_data)

# DataFrame erstellen und ausgeben
df = pd.DataFrame(sections, columns=["Abschnitt"])
print(df[:10])


                                           Abschnitt
0  Wir alle spüren es: Wie es ist, kann es nicht ...
1  Deutschlands Zukunft liegt in seinen Menschen ...
2  Wir Freie Demokraten sehen frühkindliche Bildu...
3  Nicht erst die schlechten PISA-Ergebnisse zeig...
4  Wir Freie Demokraten wollen mehr junge Mensche...
5  Deutschland muss einer der Top-Standorte für F...
6  Kultur inspiriert liberale Debattenkultur und ...
7  Unser Land hat große Stärken: Innovationskraft...
8  Deutschland steckt im Bürokratie-Burnout. Bürg...
9  Ein gerechtes, effizientes und einfaches Steue...


In [4]:
# Speichern der Tabelle als CSV-Datei
df.to_csv("/content/fdp.csv", index=False, encoding="utf-8")